In [3]:
import pandas as pd
import numpy as np

In [4]:
path_to_data = r"C:\Users\Rafay\Documents\thesis3\thesis\ActualWork\e2e\cache"
factors_list = ['RF']

# Data Load

In [5]:
def LoadData(path_to_data, e2e=True, datatype='broad'):
    if e2e:
        path_to_returns = r'{}\asset_weekly_{}.pkl'.format(path_to_data, datatype)
        path_to_prices = r'{}\assetprices_weekly_{}.pkl'.format(path_to_data, datatype)
        path_to_factors = r'{}\factor_weekly_{}.pkl'.format(path_to_data, datatype)

        returns = pd.read_pickle(path_to_returns)
        prices = pd.read_pickle(path_to_prices)
        factors = pd.read_pickle(path_to_factors)

        assets_list = prices.columns.to_list()

        returns = returns.reset_index()
        prices = prices.reset_index()
        factors = factors.reset_index()

        factors = factors.rename(columns={"Date": "date", "Mkt-RF": "RF"})
        factors = factors[['date'] + factors_list]

        return returns, assets_list, prices, factors

    path_to_prices = r'{}\prices.csv'.format(path_to_data)
    path_to_factors = r'{}\3factors.csv'.format(path_to_data)

    prices = pd.read_csv(path_to_prices)
    factors = pd.read_csv(path_to_factors)

    assets_list = list(prices['symbol'].unique())

    assets_list_cleaned = [x for x in assets_list if str(x) != 'nan']
    pivot_prices = np.round(pd.pivot_table(prices, values='close', 
                                    index='date', 
                                    columns='symbol', 
                                    aggfunc=np.mean),2)
    pivot_prices = pivot_prices.reset_index()
    pivot_prices['date'] = pd.to_datetime(pivot_prices['date'])
    factors['date'] = pd.to_datetime(factors['Date'], format="%Y%m%d")

    pivot_prices = pivot_prices.set_index('date')
    returns = pivot_prices.pct_change()
    pivot_prices = pivot_prices.reset_index()
    returns = returns.reset_index()
    returns = returns.merge(factors, on='date', how='left')
    returns = returns.drop(['Date'], axis=1)
    returns = returns.dropna()

    return returns, assets_list_cleaned, pivot_prices, []


In [6]:
datatype='cross_asset'
returns, assets_list_cleaned, prices, factors = LoadData(path_to_data, e2e=True, datatype=datatype)

In [5]:
returns.head()

,date,FTGC,BNDX,SPY
0,2014-01-03,-0.008560,-0.001609,0.003071
1,2014-01-10,0.003557,-0.004206,0.003688
2,2014-01-17,-0.000838,-0.002796,-0.013519
3,2014-01-24,-0.019560,-0.001197,0.034661
4,2014-01-31,-0.032443,-0.005337,0.022047


In [10]:
import torch

In [11]:

def DRRPW(mu,Q, delta=0):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    # Kappa
    k = 100

    # Norm for x
    p = 2

    constraints = [
        w>=0 # Disallow Short Sales
    ]

    # risk = cp.quad_form(w, Q)

    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    # We need to compute \sqrt{x^T Q x} intelligently because
    # cvxpy does not compute well with the \sqrt

    # To do this, I will take the Cholesky decomposition
    # Q = LL^T
    # Then, take the 2-norm of L*x

    # Idea: (L_1 * x_1)^2 = Q_1 x_1
    
    L = np.linalg.cholesky(Q)
    L /= np.linalg.norm(L)

    L = L.T
    
    obj = cp.power(cp.norm(L@w,2) + math.sqrt(delta)*cp.norm(w, p),2)
    obj = obj - k*log_term

    prob = cp.Problem(cp.Minimize(obj), constraints=constraints)
    
    # ECOS fails sometimes, if it does then do SCS
    try:
        prob.solve(verbose=False)
    except:
        prob.solve(solver='SCS',verbose=False)
    
    x = w.value
    x = np.divide(x, np.sum(x))
    
    # Check Risk Parity Condition is actually met
    # Note: DRRPW will not meet RP, will meet a robust version of RP
    risk_contrib = np.multiply(x, Q.dot(x))

    return x

class drrpw_custom(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, Y, delta):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        # Compute covariance
        Q = np.cov(Y.cpu().detach().numpy(), rowvar=False)

        # Get optimal allocation vector
        z_star,  = DRRPW(np.ones(Q.shape[0], Q, delta))

        # Save vectors
        ctx.save_for_backward(Q)
        ctx.save_for_backward(delta)
        ctx.save_for_backward(z_star)
        return z_star

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input

In [ ]:
asset_returns

In [12]:
drrpw = drrpw_custom()
data = torch.from_numpy(asset_returns.to_numpy())
drrpw.apply(data, 0)

: 

: 

In [8]:
lookback = 104
date = '2019-06-02'
returns_lastn = returns[(returns['date'] < date)].tail(lookback)
asset_returns = returns_lastn.drop(['date'], axis=1)

In [108]:
asset_returns

,date
287,2019-07-05
288,2019-07-12
289,2019-07-19
290,2019-07-26
291,2019-08-02
...,...
386,2021-05-28
387,2021-06-04
388,2021-06-11
389,2021-06-18


287    1562284800000000000
288    1562889600000000000
289    1563494400000000000
290    1564099200000000000
291    1564704000000000000
              ...         
386    1622160000000000000
387    1622764800000000000
388    1623369600000000000
389    1623974400000000000
390    1624579200000000000
Name: date, Length: 104, dtype: int64

In [116]:
asset_returns_tensor.size(0) - window_size + 1

53

In [1]:
from enum import Enum
import numpy as np
import cvxpy as cp
import math
from util import nearestPD, BatchUtils

import numpy as np
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable

import PortfolioClasses as pc
import LossFunctions as lf

from torch.utils.data import DataLoader

In [183]:
def drrpw_nominal_learnDelta_batched(n_y):
    # Variables
    phi = cp.Variable((n_y,1), nonneg=True)
    t = cp.Variable()
    
    # Size of uncertainty set
    delta = cp.Parameter(1, nonneg=True)

    # L - Square Root of Covariance Matrix
    L = cp.Parameter((n_y, n_y))

    # Norm for x
    p = 2

    # Kappa, dont need this to be trainable as the value of this doesnt really matter
    k = 100

    # Constraints
    constraints = [
        phi >= 0,
        t >= cp.power(cp.norm(L@phi, 2) + delta*cp.norm(phi, p),2)
        # t >= cp.power(cp.norm(L@phi, 2) + cp.norm(T@phi, 2),2)
    ]

    log_term = 0
    for i in range(n_y):
        log_term += cp.log(phi[i])

    obj = (t) - k*log_term

    # Objective function
    objective = cp.Minimize(obj)    

    # Construct optimization problem and differentiable layer
    problem = cp.Problem(objective, constraints=constraints)

    return CvxpyLayer(problem, parameters=[delta, L], variables=[phi, t])

In [207]:
from scipy.stats import gmean

def GetParameterEstimates(AssetReturns, FactorReturns, technique='OLS', log=True, bad=False, shrinkage=False):
    # Only have OLS implemented so far
    if technique!='OLS':
        return [], []
    
    if type(AssetReturns) == pd.core.frame.DataFrame:
        AssetReturns_np = AssetReturns.to_numpy()
        FactorReturns_np = FactorReturns.to_numpy()
    else:
        AssetReturns_np = AssetReturns.cpu().detach().numpy()[0]
        FactorReturns_np = FactorReturns.cpu().detach().numpy()[0][:-1]

    if shrinkage:
        Q, average_cor, shrink = GetShrinkageCov(AssetReturns_np)
        mu = 1 - (gmean(1+AssetReturns_np))

        return mu, Q

    if bad:
        Q = np.cov(AssetReturns_np, rowvar=False)
        mu = 1 - (gmean(1+AssetReturns_np))

        return mu, Q

    T,n = AssetReturns_np.shape
    _, p = FactorReturns_np.shape

    # Get Data Matrix - Factors
    X = np.zeros((T, p+1))
    X[:,:-1] = np.ones((T,1)) # Add ones to first row
    X[:,1:] = FactorReturns_np

    # Get regression coefficients for Assets
    # B = (X^TX)^(-1)X^Ty
    B = np.matmul(np.linalg.inv((np.matmul(np.transpose(X), X))), (np.matmul(np.transpose(X), AssetReturns_np)))

    # Get alpha and betas
    a = np.transpose(B[0,:])
    V = B[1:(p+1),:]

    # Residual Variance to get D
    ep = AssetReturns_np - np.matmul(X, B)
    sigma_ep = 1/(T-p-1) * np.sum(np.square(ep), axis=0)
    D = np.diag(sigma_ep)

    # Get Factor Estimated Return and Covariance Matrix
    f_bar = np.transpose(np.mean(FactorReturns_np, axis=0))
    F = np.cov(FactorReturns_np, rowvar=False)

    # Get mu
    mu = a + np.matmul(np.transpose(V), f_bar)

    # Get Q
    Q = np.matmul(np.matmul(np.transpose(V), F), V) + D

    # Make sure Q is PSD
    w,v = np.linalg.eig(Q)
    min_eig = np.min(w)


    if min_eig<0:
        print('--Not PSD--Adding Min Eigenvalue--')
        Q -= min_eig*np.identity(n)

    if log:
        print("Shape of X: {}".format(X.shape))
        print("Shape of B: {}".format(B.shape))
        print("Shape of X*B: {}".format(np.matmul(X, B).shape))
        print("Shape of ep: {}".format(ep.shape))
        print("Shape of sigma_ep: {}".format(sigma_ep.shape))
        print("Shape of D: {}".format(sigma_ep.shape))
        print("Shape of Q: {}".format(Q.shape))
    
    return mu, Q

In [193]:

def RP(mu,L):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    # Kappa
    k = 2
          
    constraints = [
        w>=0 # Disallow Short Sales
    ]

    # Objective Function
    risk = cp.norm(L@w,2)
    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    prob = cp.Problem(cp.Minimize(risk-(k*log_term)), constraints=constraints)
    
    # ECOS fails sometimes, if it does then do SCS
    try:
        prob.solve(verbose=False)
    except:
        prob.solve(solver='SCS',verbose=False)

    x = w.value
    x = np.divide(x, np.sum(x))

    return x

In [15]:
class BatchUtils:
    def __init__(self) -> None:
        pass

    def convert_to_sw_batched(self, input_numpy, window_size):
        # convert numpy array to tensor
        input_tensor = torch.from_numpy(input_numpy.to_numpy())

        # calculate number of training points
        num_training_points = input_tensor.size(0) - window_size + 1

        # initialize tensor to hold sliding windows
        sliding_windows = torch.zeros((num_training_points, window_size, input_tensor.size(1)))

        # populate tensor with sliding windows
        for i in range(num_training_points):
            sliding_windows[i] = input_tensor[i:i+window_size]
        
        return sliding_windows

    def convert_performance_periods(self, input_numpy, window_size):
        input_tensor = torch.from_numpy(input_numpy.to_numpy())
        num_training_points = input_tensor.size(0) - window_size + 1

        # define label tensor shape
        label_shape = (num_training_points, 1, input_tensor.size(1))

        # initialize label tensor to zeros
        label_set = torch.zeros(label_shape)

        # populate label tensor with next time period returns
        for i in range(num_training_points-1):
            label_set[i:i+1] = input_tensor[i+window_size:i+window_size+1].unsqueeze(1)

        return label_set
    
    def compute_covariance_matrix(self, sliding_windows):
        num_batches, window_size, num_assets = sliding_windows.size()

        # Compute the covariance matrix for all batches simultaneously
        Q = torch.Tensor(torch.cov(sliding_windows.permute(0, 2, 1)))

        # Initialize the covariance matrix tensor
        covariance_matrix_tensor = torch.zeros((num_batches, num_assets, num_assets))

        L = torch.cholesky(Q)
        L /= torch.norm(L)

        # Repeat the Cholesky matrix for each batch
        covariance_matrix_tensor = L.unsqueeze(0).expand(num_batches, -1, -1)

        return covariance_matrix_tensor
batch_utils = BatchUtils()

In [2]:
import torch
import numpy as np

In [8]:
test =torch.tensor([[-5.5405],
        [-5.9145],
        [-5.6377]])
test2 = torch.tensor([0.])

In [27]:
phi = torch.tensor([[0.3333], 
        [0.3333], 
        [0.4354]])

In [28]:
phi.squeeze()

tensor([0.3333, 0.3333, 0.4354])

In [33]:
Sigma = torch.tensor([[ 0.6721,  0.0000,  0.0000],
        [ 0.0155,  0.1084,  0.0000],
        [ 0.0084, -0.1868,  0.7081]])
phi = torch.tensor([[0.3333],
        [0.3333],
        [0.3333]])
num_assets=3
delta=torch.tensor([0.0059], requires_grad=True)
k=2

In [ ]:
def gradient(phi):
    

In [32]:
import pandas as pd
import numpy as np
from datetime import datetime
import torch

start, end = '2016-01-01', '2022-12-31'
# start, end = '2015-01-01', '2015-01-31'
factors_list = ['RF', 'SMB', 'HML']
factors_list = ['RF']

class BatchUtils:
    def __init__(self) -> None:
        pass

    def convert_to_sw_batched(self, input_numpy, window_size):
        # convert numpy array to tensor
        input_tensor = torch.from_numpy(input_numpy.to_numpy())

        # calculate number of training points
        num_training_points = input_tensor.size(0) - window_size + 1

        # initialize tensor to hold sliding windows
        sliding_windows = torch.zeros((num_training_points, window_size, input_tensor.size(1)))

        # populate tensor with sliding windows
        for i in range(num_training_points):
            sliding_windows[i] = input_tensor[i:i+window_size]
        
        return sliding_windows

    def convert_performance_periods(self, input_numpy, window_size):
        input_tensor = torch.from_numpy(input_numpy.to_numpy())
        num_training_points = input_tensor.size(0) - window_size + 1

        # define label tensor shape
        label_shape = (num_training_points, input_tensor.size(1), 1)

        # initialize label tensor to zeros
        label_set = torch.zeros(label_shape)

        # populate label tensor with next time period returns
        for i in range(num_training_points-1):
            label_set[i:i+1] = input_tensor[i+window_size:i+window_size+1].unsqueeze(2)

        return label_set
    
    def compute_covariance_matrix(self, sliding_windows):
        # get the number of batches and number of assets
        num_batches, window_size, num_assets = sliding_windows.size()

        # initialize covariance matrix tensor
        covariance_matrix_tensor = torch.zeros((num_batches, num_assets, num_assets))

        # compute covariance matrix for each batch
        for i in range(num_batches):
            batch = sliding_windows[i]
            Q = torch.Tensor(np.cov(batch.numpy().T))
            try:
                L = np.linalg.cholesky(Q)
            except:
                Q = nearestPD(Q)
                L = np.linalg.cholesky(Q)
            L /= np.linalg.norm(L)
            covariance_matrix_tensor[i] = torch.from_numpy(L)

        return covariance_matrix_tensor

def LoadData(path_to_data, e2e=True, datatype='broad'):
    if e2e:
        path_to_returns = r'{}\asset_weekly_{}.pkl'.format(path_to_data, datatype)
        path_to_prices = r'{}\assetprices_weekly_{}.pkl'.format(path_to_data, datatype)
        path_to_factors = r'{}\factor_weekly_{}.pkl'.format(path_to_data, datatype)

        returns = pd.read_pickle(path_to_returns)
        prices = pd.read_pickle(path_to_prices)
        factors = pd.read_pickle(path_to_factors)

        assets_list = prices.columns.to_list()

        returns = returns.reset_index()
        prices = prices.reset_index()
        factors = factors.reset_index()

        factors = factors.rename(columns={"Date": "date", "Mkt-RF": "RF"})
        factors = factors[['date'] + factors_list]

        return returns, assets_list, prices, factors

    path_to_prices = r'{}\prices.csv'.format(path_to_data)
    path_to_factors = r'{}\3factors.csv'.format(path_to_data)

    prices = pd.read_csv(path_to_prices)
    factors = pd.read_csv(path_to_factors)

    assets_list = list(prices['symbol'].unique())

    assets_list_cleaned = [x for x in assets_list if str(x) != 'nan']
    pivot_prices = np.round(pd.pivot_table(prices, values='close', 
                                    index='date', 
                                    columns='symbol', 
                                    aggfunc=np.mean),2)
    pivot_prices = pivot_prices.reset_index()
    pivot_prices['date'] = pd.to_datetime(pivot_prices['date'])
    factors['date'] = pd.to_datetime(factors['Date'], format="%Y%m%d")

    pivot_prices = pivot_prices.set_index('date')
    returns = pivot_prices.pct_change()
    pivot_prices = pivot_prices.reset_index()
    returns = returns.reset_index()
    returns = returns.merge(factors, on='date', how='left')
    returns = returns.drop(['Date'], axis=1)
    returns = returns.dropna()

    return returns, assets_list_cleaned, pivot_prices, []

def shrinkage(returns):
    """Shrinks sample covariance matrix towards constant correlation unequal variance matrix.
    Ledoit & Wolf ("Honey, I shrunk the sample covariance matrix", Portfolio Management, 30(2004),
    110-119) optimal asymptotic shrinkage between 0 (sample covariance matrix) and 1 (constant
    sample average correlation unequal sample variance matrix).
    Paper:
    http://www.ledoit.net/honey.pdf
    Matlab code:
    https://www.econ.uzh.ch/dam/jcr:ffffffff-935a-b0d6-ffff-ffffde5e2d4e/covCor.m.zip
    Special thanks to Evgeny Pogrebnyak https://github.com/epogrebnyak
    :param returns:
        t, n - returns of t observations of n shares.
    :return:
        Covariance matrix, sample average correlation, shrinkage.
    """
    t, n = returns.shape
    mean_returns = np.mean(returns, axis=0, keepdims=True)
    returns -= mean_returns
    sample_cov = returns.transpose() @ returns / t

    # sample average correlation
    var = np.diag(sample_cov).reshape(-1, 1)
    sqrt_var = var ** 0.5
    unit_cor_var = sqrt_var * sqrt_var.transpose()
    average_cor = ((sample_cov / unit_cor_var).sum() - n) / n / (n - 1)
    prior = average_cor * unit_cor_var
    np.fill_diagonal(prior, var)

    # pi-hat
    y = returns ** 2
    phi_mat = (y.transpose() @ y) / t - sample_cov ** 2
    phi = phi_mat.sum()

    # rho-hat
    theta_mat = ((returns ** 3).transpose() @ returns) / t - var * sample_cov
    np.fill_diagonal(theta_mat, 0)
    rho = (
        np.diag(phi_mat).sum()
        + average_cor * (1 / sqrt_var @ sqrt_var.transpose() * theta_mat).sum()
    )

    # gamma-hat
    gamma = np.linalg.norm(sample_cov - prior, "fro") ** 2

    # shrinkage constant
    kappa = (phi - rho) / gamma
    shrink = max(0, min(1, kappa / t))

    # estimator
    sigma = shrink * prior + (1 - shrink) * sample_cov

    return sigma, average_cor, shrink

def generate_date_list(data, data2, start, end):
    start = datetime.fromisoformat(start)
    end = datetime.fromisoformat(end)

    # Must be in this list
    must = data2.date.apply(lambda x: x.date()).unique().tolist()

    # Train model from start_date to date
    mask = (data['date'] >= start) & (data['date'] <= end) & data['date'].isin(must)

    data = data.loc[mask]
    return data.date.apply(lambda x: x.date()).unique().tolist()

def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = np.linalg.cholesky(B)
        return True
    except np.linalg.LinAlgError:
        return False

def nearestPD(A):
    """Find the nearest positive-definite matrix to input

    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].

    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd

    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = np.linalg.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(np.linalg.norm(A))
    # The above is different from [1]. It appears that MATLAB's `chol` Cholesky
    # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
    # Numpy's will not. So where [1] uses `eps(mineig)` (where `eps` is Matlab
    # for `np.spacing`), we use the above definition. CAVEAT: our `spacing`
    # will be much larger than [1]'s `eps(mineig)`, since `mineig` is usually on
    # the order of 1e-16, and `eps(1e-16)` is on the order of 1e-34, whereas
    # `spacing` will, for Gaussian random matrixes of small dimension, be on
    # othe order of 1e-16. In practice, both ways converge, as the unit test
    # below suggests.
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(np.linalg.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

In [9]:
t = torch.tensor([ 802.8609,  706.1788,  742.6114, -472.8843])[:3]

In [172]:
def DRRPW(mu,Q, delta=0):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    # Kappa
    k = 100

    # Norm for x
    p = 2

    constraints = [
        w>=0 # Disallow Short Sales
    ]

    # risk = cp.quad_form(w, Q)

    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    # We need to compute \sqrt{x^T Q x} intelligently because
    # cvxpy does not compute well with the \sqrt

    # To do this, I will take the Cholesky decomposition
    # Q = LL^T
    # Then, take the 2-norm of L*x

    # Idea: (L_1 * x_1)^2 = Q_1 x_1
    # Q = nearestPD(Q)
    L = Q.T

    obj = cp.power(cp.norm(L@w,2) + math.sqrt(delta)*cp.norm(w, p),2)
    obj = obj - k*log_term

    prob = cp.Problem(cp.Minimize(obj), constraints=constraints)
    
    # ECOS fails sometimes, if it does then do SCS
    try:
        prob.solve(verbose=False)
    except:
        prob.solve(solver='SCS',verbose=False)
    
    x = w.value
    x = np.divide(x, np.sum(x))
    
    # Check Risk Parity Condition is actually met
    # Note: DRRPW will not meet RP, will meet a robust version of RP
    risk_contrib = np.multiply(x, Q.dot(x))

    return x
def RP(mu,Q):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    # Kappa
    k = 2
          
    constraints = [
        w>=0 # Disallow Short Sales
    ]

    L = Q

    # L = np.linalg.cholesky(Q)
    # L /= np.linalg.norm(L)

    # L = L.T

    # Objective Function
    risk = cp.norm(L@w,2)
    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    prob = cp.Problem(cp.Minimize(risk-(k*log_term)), constraints=constraints)
    
    # ECOS fails sometimes, if it does then do SCS
    try:
        prob.solve(verbose=False)
    except:
        prob.solve(solver='SCS',verbose=False)

    x = w.value
    x = np.divide(x, np.sum(x))

    # Check Risk Parity Condition is actually met
    risk_contrib = np.multiply(x, Q.dot(x))
    if not np.all(np.isclose(risk_contrib, risk_contrib[0])):
        print("RP did not work")

    return x



In [30]:
import cvxpy as cp
import numpy as np
import numpy as np
from scipy.stats import chisquare
from scipy.stats import gmean
import cvxopt as opt
from cvxopt import matrix, spmatrix, sparse
from cvxopt.solvers import qp, options
from cvxopt import blas
import pandas as pd
options['show_progress'] = False
options['feastol'] = 1e-9

In [92]:
torch.ones((3,1))

In [296]:
class drrpw_newton_util:
    def __init__(self):
        self.tol = 1e-5
        self.max_it = 5000
        self.tol_barrier = 1e-5

    def f(self,vars, Sigma, num_assets, delta, k):
        phi = vars[:num_assets]
        lmda = vars[-1]

        return ((phi.T @ Sigma @ phi)**0.5 + delta*np.linalg.norm(phi))**2 - k*np.sum(torch.log(phi)) + lmda*(torch.ones(num_assets) @ phi - 1)
    
    def gradient(self,vars, Sigma, num_assets, delta, k):
        phi = vars[:num_assets]
        lmda = vars[-1]
        
        Sigma = Sigma.to(torch.float64)
        phi = phi.to(torch.float64)

        risk = phi.T @ Sigma @ phi
        norm = np.linalg.norm(phi)
        grad_phi = 2*(Sigma@phi + delta*risk/norm*phi + delta*norm/risk*(Sigma @ phi) + (delta**2)*phi) - k/phi

        # grad_phi = Sigma@phi - k/phi
        return grad_phi
    
    def hessian(self, vars, Sigma, num_assets, delta, k):
        phi = vars[:num_assets]
        Sigma = Sigma.to(torch.float64)
        phi = phi.to(torch.float64)

        norm = np.linalg.norm(phi)
        risk = phi.T @ Sigma @ phi
        sigma_phi = Sigma @ phi

        B = 1/(norm*((risk)**0.5))*(sigma_phi @ phi.T) - norm/((risk)**1.5)*(sigma_phi @ (sigma_phi).T) + norm/((risk)**0.5)*Sigma
        A_mat = (phi @ (sigma_phi).T)/(norm*((risk)**0.5)) + ((risk)**0.5)/norm*torch.eye(num_assets,num_assets) - ((risk)**0.5)/(norm**3) * (phi @ phi.T)

        H = 2*Sigma + 2*(delta*A_mat) + 2*(delta * B) + k*torch.diag(1 / (phi.squeeze()**2)) + 2*(delta ** 2)*torch.eye(num_assets,num_assets)
        # H = Sigma + k*torch.diag(1 / (phi.squeeze()**2))
        print('-- Hessian --')
        print(H)
        return H
        
util = drrpw_newton_util()
Sigma = torch.tensor([[ 0.7359, -0.0225,  0.1200],
        [-0.0225,  0.1269,  0.0244],
        [ 0.1200,  0.0244,  0.6167]])
phi = torch.tensor([[0.3333],
        [0.3333],
        [0.3333], [1.]])

phi = phi[:3]

In [ ]:
from time import time

-1497394.2259473808

In [297]:
num_assets = 3
alpha = 0.5
beta = 0.9
delta = 1500

print(DRRPW(np.ones(num_assets), Sigma.detach().cpu().numpy(), delta=delta))
print(RP(np.ones(num_assets), Sigma.detach().cpu().numpy()))

# Newton's method to get grad(phi) = 0.
delta = (delta)
k=1+delta

for it in range(util.max_it):
    if it>0:
        print("Iteration {}. g: {}. M: {}. M@g: {}. phi: {}".format(it, g, M, M@g, phi))
        if torch.linalg.norm(g) <= util.tol:
            break
    
        if torch.any(torch.isnan(phi)):
            break
    else:
        print("Iteration: {}. phi: {}".format(it, phi))
    
    # Source: Boyd Convex Optimization
    # Video: https://www.youtube.com/watch?v=2lPT8j4D1x0
    g = util.gradient(phi, Sigma@Sigma.T, num_assets, delta, k)
    H = util.hessian(phi, Sigma@Sigma.T, num_assets, delta, k)

    M = torch.linalg.inv(H) # TODO replace this with block inverse formulae

    # Add h(x) to bottom of gradient tensor
    g = g.to(torch.float64)
    Sigma = Sigma.to(torch.float64)
    phi = phi.to(torch.float64)
    # g = torch.cat((g, (torch.ones(num_assets).to(torch.float64)@phi[:num_assets] - 1 + 2*phi[-1].item()).unsqueeze(1)), dim=0)
    phi -= (M@g)

print("Final phi: {}".format(phi[:3]/sum(phi[:3])))

[0.33156988 0.33603059 0.33239953]
RP did not work
[0.12529569 0.75387158 0.12083273]
Iteration: 0. phi: tensor([[0.3333],
        [0.3333],
        [0.3333]])
-- Hessian --
tensor([[4.5178e+06, 3.2311e+02, 6.0319e+02],
        [3.2311e+02, 4.5150e+06, 2.7961e+02],
        [6.0319e+02, 2.7961e+02, 4.5172e+06]], dtype=torch.float64)
Iteration 1. g: tensor([[1498420.9661],
        [1495659.9150],
        [1497900.5798]], dtype=torch.float64). M: tensor([[ 2.2135e-07, -1.5839e-11, -2.9556e-11],
        [-1.5839e-11,  2.2149e-07, -1.3708e-11],
        [-2.9556e-11, -1.3708e-11,  2.2138e-07]], dtype=torch.float64). M@g: tensor([[0.3316],
        [0.3312],
        [0.3315]], dtype=torch.float64). phi: tensor([[0.0017],
        [0.0021],
        [0.0018]], dtype=torch.float64)
-- Hessian --
tensor([[5.2706e+08, 4.4445e+02, 5.5185e+02],
        [4.4445e+02, 3.5279e+08, 3.9638e+02],
        [5.5185e+02, 3.9638e+02, 4.8544e+08]], dtype=torch.float64)
Iteration 2. g: tensor([[-875038.6699],
     

In [131]:
t = torch.tensor([[ 1.9552],
        [15.0767],
        [ 3.3269]])
t/torch.sum(t)

tensor([[0.0960],
        [0.7405],
        [0.1634]])

# Old

In [ ]:
class drrpw_newton_util:
    def __init__(self):
        self.tol = 1e-5
        self.max_it = 10
        self.tol_barrier = 1e-5

    def f(self,vars, Sigma, num_assets, delta, k):
        phi = vars[:num_assets]
        lmda = vars[-1]

        return ((phi.T @ Sigma @ phi)**0.5 + delta*np.linalg.norm(phi))**2 - k*np.sum(torch.log(phi)) + lmda*(torch.ones(num_assets) @ phi - 1)
    
    def gradient(self,vars, Sigma, num_assets, delta, k):
        phi = vars[:num_assets]
        lmda = vars[-1]
        
        Sigma = Sigma.to(torch.float64)
        phi = phi.to(torch.float64)

        risk = phi.T @ Sigma @ phi
        norm = np.linalg.norm(phi)
        grad_phi = 2*(Sigma@phi + delta*risk/norm*phi + delta*norm/risk*(Sigma @ phi) + (delta**2)*phi) - k/phi + lmda.item()*torch.ones((num_assets, 1))
        return grad_phi
    
    def hessian(self, vars, Sigma, num_assets, delta, k):
        phi = vars[:num_assets]
        Sigma = Sigma.to(torch.float64)
        phi = phi.to(torch.float64)

        norm = np.linalg.norm(phi)
        risk = phi.T @ Sigma @ phi
        sigma_phi = Sigma @ phi

        B = 1/(norm*((risk)**0.5))*(sigma_phi @ phi.T) - norm/((risk)**1.5)*(sigma_phi @ (sigma_phi).T) + norm/((risk)**0.5)*Sigma
        A_mat = (phi @ (sigma_phi).T)/(norm*((risk)**0.5)) + ((risk)**0.5)/norm*torch.eye(num_assets,num_assets) - ((risk)**0.5)/(norm**3) * (phi @ phi.T)

        H = 2*Sigma + 2*(delta*A_mat) + 2*(delta * B) + k*torch.diag(1 / (phi.squeeze()**2)) + 2*(delta ** 2)*torch.eye(num_assets,num_assets)
        
        return H
            for it in range(util.max_it):
                if it>0:
                    print("Iteration {}. g: {}. M: {}. M@g: {}. phi: {}".format(it, g, M, M@g, phi))
                else:
                    print("Iteration: {}. phi: {}".format(it, phi))
                
                if torch.linalg.norm(g) <= util.tol:
                    break
                
                if torch.any(torch.isnan(phi)):
                    break

                # Source: Boyd Convex Optimization
                # Video: https://www.youtube.com/watch?v=2lPT8j4D1x0
                g = util.gradient(phi, Sigma@Sigma.T, num_assets, delta, k)
                H = util.hessian(phi, Sigma@Sigma.T, num_assets, delta, k)

                ones = torch.ones(num_assets, 1)  # Column vector of ones

                ones = torch.ones(num_assets, 1)  # Column vector of ones
                zero = torch.tensor([0.])  # Scalar zero

                # Concatenate the blocks to form the block matrix M
                top_block = torch.cat((H, ones), dim=1)
                bottom_block = torch.cat((ones, zero.unsqueeze(1)), dim=0).t()
                H_constrained = torch.cat((top_block, bottom_block), dim=0)
                M = torch.linalg.inv(H_constrained) # TODO replace this with block inverse formulae

                # Add h(x) to bottom of gradient tensor
                g = g.to(torch.float64)
                Sigma = Sigma.to(torch.float64)
                phi = phi.to(torch.float64)
                g = torch.cat((g, (torch.ones(num_assets).to(torch.float64)@phi[:num_assets] - 1).unsqueeze(1)), dim=0)
                phi -= (M@g)


In [61]:
for it in range(1000):
    if torch.linalg.norm(g) <= 1e-6:
        break

    if it>0:
        print("New newton step: {}. Current phi: {}. M: {}. grad: {}. M@grad: {}".format(it, phi, M, g, M@g))
    
    if torch.any(torch.isnan(phi)):
        break

    g = gradient(phi)
    H = hessian(phi)

    M = torch.linalg.inv(H)

    phi -= (beta)*(M@g)

tensor([[-7.4506e-09],
        [ 5.7075e-04],
        [-5.7074e-04],
        [-5.7075e+00]])

In [58]:
g = torch.tensor([[-5.7075],
        [-5.7075],
        [-5.7075],
        [ 0.0000]])

In [60]:
M = torch.tensor([[ 0.0341, -0.0185, -0.0156,  0.3087],
        [-0.0182,  0.0367, -0.0186,  0.3679],
        [-0.0159, -0.0182,  0.0342,  0.3234],
        [ 0.3079,  0.3754,  0.3167, -6.2706]])

In [63]:
g.size()

torch.Size([4, 1])

M.size()

In [36]:
risk = phi.T @ Sigma @ phi
norm = np.linalg.norm(phi)

In [13]:
# To do list
'''
    1. Re-do T and T Diagonal
    2. Re-do MVO Norm Learning
    3. 
'''

In [15]:
T = np.array([[ 4.30433092e+00, -8.36058512e-04, -1.33765299e-02],
 [ 2.78223877e+00,  2.09977763e-01, -9.87433303e-01],
 [ 3.17641391e+00, -6.57556446e-01,  1.22038769e+00]])

0.477716704732

In [12]:
x = torch.randn((5,5))
print(x)
print(x.size())
x = x.repeat(53,1,1)
print(x[9])

tensor([[ 0.3008,  1.1426,  0.0753,  1.0837, -0.3898],
        [-0.6324,  0.7153, -0.5822,  0.4831,  1.1137],
        [ 0.9963, -1.9636, -0.2414, -1.0335, -0.3084],
        [ 0.0211, -0.9552,  0.9934,  2.3207, -0.4519],
        [-0.1446,  0.2275,  0.7958,  0.1948, -0.0314]])
torch.Size([5, 5])
tensor([[ 0.3008,  1.1426,  0.0753,  1.0837, -0.3898],
        [-0.6324,  0.7153, -0.5822,  0.4831,  1.1137],
        [ 0.9963, -1.9636, -0.2414, -1.0335, -0.3084],
        [ 0.0211, -0.9552,  0.9934,  2.3207, -0.4519],
        [-0.1446,  0.2275,  0.7958,  0.1948, -0.0314]])


In [258]:
date = '2021-01-10'

In [16]:
lookback = 104
returns_lastn = returns[(returns['date'] < date)].tail(lookback)
print()
asset_returns = returns_lastn.drop(['date'], axis=1)
window_size = 52
batched_tensor = batch_utils.convert_to_sw_batched(asset_returns, window_size)
performance_tensor = batch_utils.convert_performance_periods(asset_returns, window_size)
Q_batched = batch_utils.compute_covariance_matrix(batched_tensor)

RuntimeError: cov(): expected input to have two or fewer dimensions but got an input with 3 dimensions

In [271]:
test = [0,1,2,3,4,5,6]
window_size = 3
training_pts = len(test) - window_size + 1

training = []
for i in range(training_pts):
    training[i] = test[i:i+window_size]

testing = []
for i in range(training_pts-1):
    testing[i:i+1] = test[i+window_size:i+window_size+1]

IndexError: list assignment index out of range

In [279]:
performance_tensor[9]

tensor([[0.0400, 0.0141, 0.0692]])

In [280]:
batched_tensor[10][-1]

tensor([0.0400, 0.0141, 0.0692])

In [260]:
lookback = 52
returns_lastn = returns[(returns['date'] < date)].tail(lookback)
asset_returns = returns_lastn.drop(['date'], axis=1)
mu, Q = GetParameterEstimates(asset_returns, asset_returns, log=False, bad=True, shrinkage=False)

In [261]:
Q_batched[-1]

tensor([[0.5082, 0.0000, 0.0000],
        [0.0501, 0.1272, 0.0000],
        [0.6650, 0.1414, 0.5107]])

In [262]:
L = np.linalg.cholesky(Q)
L /= np.linalg.norm(L)
L

array([[0.50819596, 0.        , 0.        ],
       [0.05012327, 0.12720189, 0.        ],
       [0.66497918, 0.14144394, 0.51072548]])

In [234]:
Q_batched = torch.tensor(Q)
Q_batched.unsqueeze(1).size()

torch.Size([3, 1, 3])

In [263]:
# DRRPW
z_drrpw = DRRPW(mu, Q)

# Learn Delta
L = np.linalg.cholesky(Q)
L /= np.linalg.norm(L)
Q_batched = torch.tensor(L)
Q_batched = Q_batched.unsqueeze(0)
Q_batched = Q_batched.type(torch.FloatTensor)
batch_size, num_assets, num_assets = Q_batched.size()
opt_layer = drrpw_nominal_learnDelta_batched(num_assets)
batched_delta = torch.ones((batch_size,1))*0
batched_delta = batched_delta.type(torch.FloatTensor)
z_stars, _ = opt_layer(batched_delta, Q_batched)

# Make each portfolio sum to 1
sums = z_stars.sum(dim=1, keepdim=True)
z_stars = z_stars / sums

# RP
z_rp = RP(mu, Q)

RP did not work


In [264]:
print("DRRPW: {}".format(z_drrpw))
print("Learn Delta: {}".format(z_stars))
print("RP: {}".format(z_rp))

DRRPW: [0.14172789 0.63553458 0.22273754]
Learn Delta: tensor([[[0.1417],
         [0.6355],
         [0.2227]]])
RP: [0.14172835 0.63553333 0.22273832]


In [202]:
def RP(mu,Q):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    # Kappa
    k = 2
          
    constraints = [
        w>=0 # Disallow Short Sales
    ]

    L = np.linalg.cholesky(Q)
    L /= np.linalg.norm(L)

    # Objective Function
    risk = cp.norm(L@w,2)
    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    prob = cp.Problem(cp.Minimize(risk-(k*log_term)), constraints=constraints)
    
    # ECOS fails sometimes, if it does then do SCS
    try:
        prob.solve(verbose=False)
    except:
        prob.solve(solver='SCS',verbose=False)

    x = w.value
    x = np.divide(x, np.sum(x))

    # Check Risk Parity Condition is actually met
    risk_contrib = np.multiply(x, Q.dot(x))
    if not np.all(np.isclose(risk_contrib, risk_contrib[0])):
        print("RP did not work")

    return x

'''
Distributionally Robust Risk Parity With Wasserstein Distance Optimizer
Inputs: mu: numpy array, key: Symbol. value: return estimate
        Q: nxn Asset Covariance Matrix (n: # of assets)
        delta: size of the uncertainty set
Outputs: x: optimal allocations

Formula:
    \min_{\boldsymbol{\phi} \in \mathcal{X}} {(\sqrt{\boldsymbol{\phi}^T \Sigma_{\mathcal{P}}(R)\boldsymbol{\phi}} + \sqrt{\delta}||\boldsymbol{\phi}||_p)^2} - c\sum_{i=1}^n ln(y)

'''

def DRRPW(mu,Q, delta=0):
    
    # # of Assets
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    # Kappa
    k = 100

    # Norm for x
    p = 2

    constraints = [
        w>=0 # Disallow Short Sales
    ]

    # risk = cp.quad_form(w, Q)

    log_term = 0
    for i in range(n):
        log_term += cp.log(w[i])
    
    # We need to compute \sqrt{x^T Q x} intelligently because
    # cvxpy does not compute well with the \sqrt

    # To do this, I will take the Cholesky decomposition
    # Q = LL^T
    # Then, take the 2-norm of L*x

    # Idea: (L_1 * x_1)^2 = Q_1 x_1
    
    L = np.linalg.cholesky(Q)
    L /= np.linalg.norm(L)
    
    obj = cp.power(cp.norm(L@w,2) + math.sqrt(delta)*cp.norm(w, p),2)
    obj = obj - k*log_term

    prob = cp.Problem(cp.Minimize(obj), constraints=constraints)
    
    # ECOS fails sometimes, if it does then do SCS
    try:
        prob.solve(verbose=False)
    except:
        prob.solve(solver='SCS',verbose=False)
    
    x = w.value
    x = np.divide(x, np.sum(x))
    
    # Check Risk Parity Condition is actually met
    # Note: DRRPW will not meet RP, will meet a robust version of RP
    risk_contrib = np.multiply(x, Q.dot(x))

    return x


def drrpw_nominal_learnDelta(Q):
    n_y = Q.shape[-1]
    # Variables
    phi = cp.Variable((n_y,1), nonneg=True)
    t = cp.Variable()
    
    # Size of uncertainty set
    delta = cp.Parameter(nonneg=True)
    # T = cp.Parameter((n_y, n_y), PSD=True)

    # Norm for x. TODO set this to be the Mahalanobis Norm
    p = 2

    # Kappa, dont need this to be trainable as the value of this doesnt really matter
    k = 2

    # We need to compute \sqrt{x^T Q x} intelligently because
    # cvxpy does not compute well with the \sqrt

    # To do this, I will take the Cholesky decomposition
    # Q = LL^T
    # Then, take the 2-norm of L*x

    # Idea: (L_1 * x_1)^2 = Q_1 x_1

    try:
        L = np.linalg.cholesky(Q)
    except:
        Q = nearestPD(Q)
        L = np.linalg.cholesky(Q)

    L /= np.linalg.norm(L)

    # Constraints
    constraints = [
        phi >= 0,
        t >= cp.power(cp.norm(L@phi, 2) + delta*cp.norm(phi, p),2)
        # t >= cp.power(cp.norm(L@phi, 2) + cp.norm(T@phi, 2),2)
    ]

    log_term = 0
    for i in range(n_y):
        log_term += cp.log(phi[i])

    # obj = cp.power(cp.norm(L@w, 2) + delta*cp.norm(w, p),2)
    # obj = cp.sum_squares(cp.norm(L@w, 2) + delta*cp.norm(w, p))
    # cp.quad_form(w, Q)
    # obj = cp.quad_form(w, Q) + 2*delta*cp.norm(w,2)*cp.norm(L@w, 2) + cp.norm(w,2)
    # print('using this one')
    # obj = 2*delta*cp.norm(w,2)*cp.norm(L@w_tilde, 2)
    obj = (t) - k*log_term

    # Objective function
    objective = cp.Minimize(obj)    

    # Construct optimization problem and differentiable layer
    problem = cp.Problem(objective, constraints=constraints)

    return CvxpyLayer(problem, parameters=[delta], variables=[phi, t])


def drrpw_nominal_learnDelta_batched(n_y):
    # Variables
    phi = cp.Variable((n_y,1), nonneg=True)
    t = cp.Variable()
    
    # Size of uncertainty set
    delta = cp.Parameter(1, nonneg=True)

    # L - Square Root of Covariance Matrix
    L = cp.Parameter((n_y, n_y))

    # Norm for x
    p = 2

    # Kappa, dont need this to be trainable as the value of this doesnt really matter
    k = 2

    # Constraints
    constraints = [
        phi >= 0,
        t >= cp.power(cp.norm(L@phi, 2) + delta*cp.norm(phi, p),2)
        # t >= cp.power(cp.norm(L@phi, 2) + cp.norm(T@phi, 2),2)
    ]

    log_term = 0
    for i in range(n_y):
        log_term += cp.log(phi[i])

    obj = (t) - k*log_term

    # Objective function
    objective = cp.Minimize(obj)    

    # Construct optimization problem and differentiable layer
    problem = cp.Problem(objective, constraints=constraints)

    return CvxpyLayer(problem, parameters=[delta, L], variables=[phi, t])

In [201]:
indx = 0
layer = drrpw_nominal_learnDelta(Q[indx])
delta = torch.zeros(1)
z_star, _ = layer(delta)
# Make each portfolio sum to 1
sums = z_star.sum(dim=1, keepdim=True)
z_stars = z_star / sums
print('Forward Pass: {}'.format(z_stars))
print('RP: {}'.format(RP([0]*3, Q[indx])))
print('DRRPW: {}'.format(DRRPW([0]*3, Q[indx])))

Forward Pass: tensor([[[4.3807e-09],
         [1.0000e+00],
         [1.8199e-06]]])
RP: [0.22014154 0.64599017 0.13386829]
DRRPW: [0.03367387 0.94202728 0.02429885]


In [66]:
DRRPW(None, Q, delta=0.0016)

array([0.09873581, 0.78478117, 0.11648302])

In [11]:
def compute_covariance_matrix(sliding_windows):
    # get the number of batches and number of assets
    num_batches, window_size, num_assets = sliding_windows.size()

    # initialize covariance matrix tensor
    covariance_matrix_tensor = torch.zeros((num_batches, num_assets, num_assets))

    # compute covariance matrix for each batch
    for i in range(num_batches):
        batch = sliding_windows[i]
        covariance_matrix = torch.Tensor(np.cov(batch.numpy().T))
        covariance_matrix_tensor[i] = covariance_matrix

    return covariance_matrix_tensor


In [12]:
compute_covariance_matrix(sliding_windows)

: 

: 

In [5]:
%pip install ics


     ---------------------------------------- 40.1/40.1 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 101.5/101.5 kB 5.7 MB/s eta 0:00:00


In [23]:
import ics
import datetime

In [50]:
from ics import Calendar, Event

In [57]:
i = 0
c = Calendar()
for week in range(15):
    for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
        date = date_list[i]
        # print(plan.loc[plan['WEEK'] == float(week)])
        do = plan.loc[plan['WEEK'] == (week+1)][day].values[0]
        i+=1

        e = Event(name="HM Training Day {}".format(i), begin=date, description=do)
        c.events.add(e)    

In [58]:
with open('my.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())

In [22]:
plan = plan.dropna(subset='WEEK')
plan.head(15)

,WEEK,Date,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Weekly Mileage,Unnamed: 10
0,1.0,NaN,Rest Day,Walk / Run 10 x \n2 min walk\n1 min run,Rest Day,Walk / Run 10 x \n2 min walk\n1 min run,Strength Training 45-60 mins,Rest Day,Long Run 2.5k (walk if needed),Around\n9k,5K PHASE
3,2.0,NaN,Rest Day,Walk / Run 15 x \n1 min walk\n1 min run,Rest Day,Walk / Run 15 x \n1 min walk\n1 min run,Strength Training 45-60 mins,Walk / Run 15 x \n1 min walk\n1 min run,Long Run 3k (walk if needed),Around\n13k,NaN
6,3.0,NaN,Rest Day,Walk / Run 15 x \n0.5 min walk\n1.5 min run,Rest Day,Walk / Run 15 x \n0.5 min walk\n1.5 min run,Strength Training 45-60 mins,Walk / Run 15 x \n0.5 min walk\n1.5 min run,Long Run 4k (walk if needed),Around\n15k,NaN
9,4.0,NaN,Rest Day,Walk / Run 10 x \n1 min walk\n2 min run,Rest Day,Walk / Run 10 x \n1 min walk\n2 min run,Strength Training 45-60 mins,Rest Day,5 k,Around\n13k,NaN
12,5.0,NaN,Rest Day,4 k,3 k,Rest Day,4 k,Strength Training,6 k,17 k,10K PHASE
15,6.0,NaN,Rest Day,4 k,3 k,Rest Day,4 k,Strength Training,7 k,18 k,NaN
18,7.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,8 k,25 k,NaN
21,8.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,10 k,27 k,NaN
24,9.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,8 k,25 k,HALF MARATHON PHASE
27,10.0,NaN,Rest Day,5 k,7 k,Rest Day,5 k,Strength Training,11 k,28 k,NaN


# (i) Ledoit-Wolfe Shrinkage

In [7]:
def shrinkage(returns):
    """Shrinks sample covariance matrix towards constant correlation unequal variance matrix.
    Ledoit & Wolf ("Honey, I shrunk the sample covariance matrix", Portfolio Management, 30(2004),
    110-119) optimal asymptotic shrinkage between 0 (sample covariance matrix) and 1 (constant
    sample average correlation unequal sample variance matrix).
    Paper:
    http://www.ledoit.net/honey.pdf
    Matlab code:
    https://www.econ.uzh.ch/dam/jcr:ffffffff-935a-b0d6-ffff-ffffde5e2d4e/covCor.m.zip
    Special thanks to Evgeny Pogrebnyak https://github.com/epogrebnyak
    :param returns:
        t, n - returns of t observations of n shares.
    :return:
        Covariance matrix, sample average correlation, shrinkage.
    """
    t, n = returns.shape
    mean_returns = np.mean(returns, axis=0, keepdims=True)
    returns -= mean_returns
    sample_cov = returns.transpose() @ returns / t

    # sample average correlation
    var = np.diag(sample_cov).reshape(-1, 1)
    sqrt_var = var ** 0.5
    unit_cor_var = sqrt_var * sqrt_var.transpose()
    average_cor = ((sample_cov / unit_cor_var).sum() - n) / n / (n - 1)
    prior = average_cor * unit_cor_var
    np.fill_diagonal(prior, var)

    # pi-hat
    y = returns ** 2
    phi_mat = (y.transpose() @ y) / t - sample_cov ** 2
    phi = phi_mat.sum()

    # rho-hat
    theta_mat = ((returns ** 3).transpose() @ returns) / t - var * sample_cov
    np.fill_diagonal(theta_mat, 0)
    rho = (
        np.diag(phi_mat).sum()
        + average_cor * (1 / sqrt_var @ sqrt_var.transpose() * theta_mat).sum()
    )

    # gamma-hat
    gamma = np.linalg.norm(sample_cov - prior, "fro") ** 2

    # shrinkage constant
    kappa = (phi - rho) / gamma
    shrink = max(0, min(1, kappa / t))

    # estimator
    sigma = shrink * prior + (1 - shrink) * sample_cov

    return sigma, average_cor, shrink

In [8]:
sigma, average_cor, shrink = shrinkage(returns.drop('date', axis=1).to_numpy())

In [9]:
sigma

array([[4.11719612e-04, 1.05466722e-05, 1.72681015e-04],
       [1.05466722e-05, 2.97840162e-05, 3.20162546e-05],
       [1.72681015e-04, 3.20162546e-05, 5.96583123e-04]])